# W_GL_BALANCE_FS ETL Process
### Oracle EBS GL_BALANCES to EDW W_GL_BALANCE_FS (Fact Staging)

In [ ]:
%sql
-- 1) PARAMETERS
CREATE OR REPLACE TEMP VIEW etl_params AS
SELECT CASE WHEN COUNT(*) > 0 THEN 'Y' ELSE 'N' END AS IS_INCREMENTAL,
  COALESCE(date_format(MIN(LAST_MAX_DATE), 'yyyy-MM-dd HH:mm:ss'), '1970-01-01 00:00:00') AS LAST_EXTRACT_DATE,
  380 AS DATASOURCE_NUM_ID, '__NOT_APPLICABLE__' AS ETL_USAGE_CODE,
  '1' AS TENANT_ID, 2623812 AS ETL_PROC_WID, 12345 AS EXECUTION_ID, 30 AS PRUNE_DAYS
FROM workspace.oracle_edw.w_etl_load_dates
WHERE PACKAGE_NAME = 'SDE_ORAR122_ADAPTOR_SDE_ORA_GLBALANCEFACT'
  AND DATASOURCE_NUM_ID = 380 AND ETL_USAGE_CODE = '__NOT_APPLICABLE__' AND COMMITTED = '1';

In [ ]:
%sql
-- 2) STAGING: Extract GL balances with complex transformations
CREATE OR REPLACE TEMP VIEW stg_gl_balances AS
SELECT
  glb.ACTUAL_FLAG,
  CASE WHEN glb.ACTUAL_FLAG = 'B' THEN 'PTD' ELSE 'YTD' END AS BALANCE_TYPE_INDICATOR,
  glb.PERIOD_NAME,
  gp.END_DATE,
  CAST(gp.PERIOD_NUM AS STRING) AS PERIOD_ID,
  gl.COMPLETE_FLAG AS FINANCIAL_GL_FLG,
  gl.ENABLE_BUDGETARY_CONTROL_FLAG AS BUDGETARY_CONTROL_FLG,
  gl.CURRENCY_CODE AS LEDGER_CURRENCY_CODE,
  gp.ADJUSTMENT_PERIOD_FLAG,
  CAST(glb.CODE_COMBINATION_ID AS STRING) AS GL_ACCOUNT_ID,
  -- Balance calculations based on currency type
  CASE
    WHEN glb.CURRENCY_TYPE IN ('TRANSLATED', 'ENTERED_FOREIGN', 'STAT') THEN glb.BEGIN_BALANCE_DR - glb.BEGIN_BALANCE_CR
    ELSE COALESCE(glb.BEGIN_BALANCE_DR_BEQ - glb.BEGIN_BALANCE_CR_BEQ, 0)
  END AS BALANCE_ACCT_AMT,
  CASE
    WHEN glb.CURRENCY_TYPE IN ('BASE', 'ENTERED_LEDGER') THEN glb.BEGIN_BALANCE_DR - glb.BEGIN_BALANCE_CR
    ELSE 0
  END AS BALANCE_LOC_AMT,
  -- Activity calculations
  CASE
    WHEN glb.CURRENCY_TYPE IN ('TRANSLATED', 'ENTERED_FOREIGN', 'STAT') THEN glb.PERIOD_NET_DR - glb.PERIOD_NET_CR
    ELSE COALESCE(glb.PERIOD_NET_DR_BEQ - glb.PERIOD_NET_CR_BEQ, 0)
  END AS ACTIVITY_ACCT_AMT,
  CASE
    WHEN glb.CURRENCY_TYPE IN ('BASE', 'ENTERED_LEDGER') THEN glb.PERIOD_NET_DR - glb.PERIOD_NET_CR
    ELSE 0
  END AS ACTIVITY_LOC_AMT,
  CASE WHEN glb.CURRENCY_TYPE = 'BASE' THEN 'NULL' ELSE glb.CURRENCY_CODE END AS ACCT_CURR_CODE,
  CASE WHEN glb.TRANSLATED_FLAG IS NULL OR glb.TRANSLATED_FLAG = 'R' THEN 'N' ELSE 'Y' END AS TRANSLATED_FLAG,
  CAST(glb.LEDGER_ID AS STRING) AS LEDGER_ID,
  -- Complex integration ID construction
  CONCAT('GL~', CAST(glb.LEDGER_ID AS STRING), '~', CAST(glb.CODE_COMBINATION_ID AS STRING), '~',
    glb.PERIOD_NAME, '~', CASE WHEN glb.ACTUAL_FLAG = 'B' THEN 'PTD' ELSE 'YTD' END, '~', glb.ACTUAL_FLAG, '~',
    CASE WHEN glb.TRANSLATED_FLAG IS NULL OR glb.TRANSLATED_FLAG = 'R' THEN 'N' ELSE 'Y' END, '~',
    gl.CURRENCY_CODE, '~', CASE WHEN glb.CURRENCY_TYPE = 'BASE' THEN 'NULL' ELSE glb.CURRENCY_CODE END, '~',
    CAST(glb.TEMPLATE_ID AS STRING)) AS INTEGRATION_ID,
  glb.LAST_UPDATE_DATE AS LAST_UPDATE_DATE_BAL,
  gp.LAST_UPDATE_DATE AS LAST_UPDATE_DATE_PERIODS,
  gl.LAST_UPDATE_DATE AS LAST_UPDATE_DATE_LED
FROM workspace.oracle_ebs.GL_BALANCES glb
INNER JOIN workspace.oracle_ebs.GL_PERIODS gp
  ON glb.PERIOD_NAME = gp.PERIOD_NAME AND glb.LEDGER_ID = gp.LEDGER_ID
INNER JOIN workspace.oracle_ebs.GL_LEDGERS gl
  ON glb.LEDGER_ID = gl.LEDGER_ID
CROSS JOIN etl_params p
WHERE glb.ACTUAL_FLAG IN ('A', 'B')
  AND (glb.LAST_UPDATE_DATE > p.LAST_EXTRACT_DATE OR gp.LAST_UPDATE_DATE > p.LAST_EXTRACT_DATE OR gl.LAST_UPDATE_DATE > p.LAST_EXTRACT_DATE);

In [ ]:
%sql
-- 3) INSERT INTO W_GL_BALANCE_FS
INSERT INTO workspace.oracle_edw.w_gl_balance_fs
(
  ACTUAL_FLAG, BALANCE_TYPE_INDICATOR, PERIOD_ID, END_DATE,
  FINANCIAL_GL_FLG, BUDGETARY_CONTROL_FLG, LEDGER_CURRENCY_CODE,
  ADJUSTMENT_PERIOD_FLAG, GL_ACCOUNT_ID,
  BALANCE_ACCT_AMT, BALANCE_LOC_AMT, ACTIVITY_ACCT_AMT, ACTIVITY_LOC_AMT,
  ACCT_CURR_CODE, TRANSLATED_FLAG, LEDGER_ID,
  DATASOURCE_NUM_ID, INTEGRATION_ID, TENANT_ID
)
SELECT
  s.ACTUAL_FLAG, s.BALANCE_TYPE_INDICATOR, s.PERIOD_ID, s.END_DATE,
  s.FINANCIAL_GL_FLG, s.BUDGETARY_CONTROL_FLG, s.LEDGER_CURRENCY_CODE,
  s.ADJUSTMENT_PERIOD_FLAG, s.GL_ACCOUNT_ID,
  s.BALANCE_ACCT_AMT, s.BALANCE_LOC_AMT, s.ACTIVITY_ACCT_AMT, s.ACTIVITY_LOC_AMT,
  s.ACCT_CURR_CODE, s.TRANSLATED_FLAG, s.LEDGER_ID,
  p.DATASOURCE_NUM_ID, s.INTEGRATION_ID, p.TENANT_ID
FROM stg_gl_balances s
CROSS JOIN etl_params p;

In [ ]:
%sql
-- 4) UPDATE CONTROL (Standard pattern)
MERGE INTO workspace.oracle_edw.w_etl_load_dates AS tgt
USING (
  SELECT p.DATASOURCE_NUM_ID, 'SDE_ORAR122_ADAPTOR_SDE_ORA_GLBALANCEFACT' AS package_name,
    'W_GL_BALANCE_FS' AS target_table_name, p.ETL_USAGE_CODE, p.ETL_PROC_WID,
    p.EXECUTION_ID AS load_plan_id, current_timestamp() AS etl_load_date,
    CASE WHEN p.IS_INCREMENTAL = 'Y' THEN '1' ELSE '0' END AS committed,
    date_add(current_timestamp(), -p.PRUNE_DAYS) AS wip_load_start_date
  FROM etl_params p
) src
ON tgt.datasource_num_id = src.datasource_num_id AND tgt.package_name = src.package_name
WHEN MATCHED THEN UPDATE SET tgt.etl_load_date = src.etl_load_date
WHEN NOT MATCHED THEN INSERT VALUES (src.datasource_num_id, src.package_name, src.target_table_name,
  src.etl_usage_code, src.etl_proc_wid, src.load_plan_id, src.wip_load_start_date, NULL, src.etl_load_date, src.committed);

In [ ]:
%sql
DROP VIEW IF EXISTS stg_gl_balances;
DROP VIEW IF EXISTS etl_params;